In [1]:
import matplotlib.pyplot as plt
import pathlib
import tqdm
import itertools
import xml.etree.ElementTree as ET

In [2]:
detect_window = [48,48]
stride = 8
gt_contained_threshold = 0.3

image_paths = list(pathlib.Path(r'D:\Datasets\Golf\IdeasLab\backup\golfheadcropped').glob('*.jpg'))
bbox_path = pathlib.Path(r'D:\Datasets\Golf\IdeasLab\backup\golfheadcropped_bbox')
save_root = pathlib.Path(r'D:\Datasets\Golf\IdeasLab\clubheads')

save_root.joinpath('positive').mkdir()
save_root.joinpath('negative').mkdir()

for p in tqdm.tqdm(image_paths):
    image = plt.imread(p.as_posix())
    
    h,w = image.shape[:2]
    H = list(range(0,w-detect_window[0],stride)) + [w-detect_window[0]]
    W = list(range(0,h-detect_window[1],stride)) + [h-detect_window[1]]
    
    bbox_label = bbox_path.joinpath(p.with_suffix('.xml').name)
    if bbox_label.exists():
        xml_tree = ET.parse(bbox_label.as_posix())
        gt_xmin, gt_ymin, gt_xmax, gt_ymax = list(int(x.text) for x in xml_tree.find('object').find('bndbox'))
        gt_area = (gt_xmax-gt_xmin)*(gt_ymax-gt_ymin)
        
    else:
        xml_tree = None
        gt_xmin, gt_ymin, gt_xmax, gt_ymax = None,None,None,None
        gt_area = None
        
    image_name = p.with_suffix('').name
    
    for i, tl in enumerate(itertools.product(W,H)):
        t,l = tl
        
        cp_xmin, cp_xmax = l,l+detect_window[1]
        cp_ymin, cp_ymax = t,t+detect_window[0]
        
        if bbox_label.exists():
            
            inter_xmin = max(cp_xmin,gt_xmin)
            inter_ymin = max(cp_ymin,gt_ymin)
            inter_xmax = min(cp_xmax,gt_xmax)
            inter_ymax = min(cp_ymax,gt_ymax)
            
            inter_area = max(0,inter_xmax-inter_xmin+1) * max(0,inter_ymax-inter_ymin+1)
            
            gt_contained_ratio = inter_area/gt_area
            if gt_contained_ratio > gt_contained_threshold:
                classify = 'positive'
            else:
                classify = 'negative'
        else:
            classify = 'negative'
        save_folder = save_root.joinpath(classify)
        cropped_image = image[cp_ymin:cp_ymax,cp_xmin:cp_xmax,:]
        plt.imsave(save_folder.joinpath(f'{image_name}_{i:0>3}.jpg'),cropped_image)

100%|██████████| 343/343 [00:29<00:00, 11.79it/s]


In [8]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression,make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostRegressor,AdaBoostClassifier

# define dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, random_state=6)#, noise=0.1)
# define the model

model = AdaBoostClassifier(SVC(kernel='linear',probability=True),n_estimators=10,  learning_rate=1.0, algorithm='SAMME.R')
# evaluate the model
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: -0.206 (0.037)


In [9]:
val_x, val_y = make_classification(n_samples=10, n_features=20, n_informative=15, random_state=3)

In [11]:
model.predict(val_x)

NotFittedError: This AdaBoostClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [10]:
val_y

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 0])